# Trabalho 1 – Disciplina de IA
## Tema 6 – Alocação de Salas (Timetabling Simplificado)

**Grupo:**  
- Nome 1  
- Nome 2  
- Nome 3  

**Problema:**  
Queremos alocar disciplinas em salas e horários disponíveis, respeitando restrições como:

- Capacidade das salas (número de alunos ≤ capacidade da sala).
- Um mesmo professor não pode ministrar mais de uma disciplina no mesmo horário.
- Cada disciplina deve ser alocada em exatamente uma combinação (sala, horário).

Vamos modelar esse problema como **busca em espaço de estados**, e comparar estratégias:

- Uma busca **sem informação** (híbrido BFS + DFS).
- Uma busca **com informação** (busca gulosa com heurística).


In [1]:
# Dados de exemplo do problema de alocação de salas

salas = [
    {"id": "S1", "capacidade": 40},
    {"id": "S2", "capacidade": 30},
    {"id": "S3", "capacidade": 20},
]

horarios = ["08-10", "10-12", "14-16"]

disciplinas = [
    {"id": "IA",   "prof": "Lucidio", "alunos": 35},
    {"id": "PO",   "prof": "Maria",   "alunos": 25},
    {"id": "ALC",  "prof": "Joao",    "alunos": 20},
    {"id": "BD",   "prof": "Maria",   "alunos": 28},  # mesma prof da PO → gera conflito
    {"id": "CG",   "prof": "Ana",     "alunos": 18},
]

# Estado inicial: dicionário vazio (nada alocado)
estado_inicial = {}


## Modelagem do problema

- **Estado**: um dicionário onde a chave é o ID da disciplina e o valor é um par (sala, horário).  
  Exemplo:  
  `{"IA": ("S1", "08-10"), "PO": ("S2", "10-12")}`

- **Ação**: escolher uma disciplina ainda não alocada e tentar colocá-la em uma combinação (sala, horário).

- **Teste de objetivo**: todas as disciplinas foram alocadas e nenhuma restrição é violada.

- **Restrições que verificamos:**
  1. Capacidade da sala ≥ número de alunos da disciplina.
  2. Professor não pode dar duas disciplinas no mesmo horário.

A seguir, definimos funções em Python para:
- Encontrar disciplinas não alocadas.
- Verificar se uma alocação é válida.
- Gerar estados sucessores (novas alocações possíveis).


In [2]:
# Funções auxiliares para a modelagem do problema

def disciplinas_nao_alocadas(estado, disciplinas):
    """Retorna a lista de disciplinas que ainda não foram alocadas no estado."""
    ids_alocadas = set(estado.keys())
    return [d for d in disciplinas if d["id"] not in ids_alocadas]


def get_disciplina(disciplinas, disc_id):
    """Retorna o dicionário da disciplina dado o ID."""
    return next(d for d in disciplinas if d["id"] == disc_id)


def get_sala(salas, sala_id):
    """Retorna o dicionário da sala dado o ID."""
    return next(s for s in salas if s["id"] == sala_id)


def eh_alocacao_valida(estado, disciplinas, salas, disc_id, sala_id, horario):
    """
    Verifica se alocar a disciplina 'disc_id' na sala 'sala_id' e horário 'horario'
    é válido considerando:
      - capacidade da sala
      - conflitos de professor no mesmo horário
    """
    disc = get_disciplina(disciplinas, disc_id)
    sala = get_sala(salas, sala_id)
    
    # 1) Capacidade da sala
    if disc["alunos"] > sala["capacidade"]:
        return False
    
    # 2) Conflito de professor no mesmo horário
    for d_id, (s_id, h) in estado.items():
        if h == horario:
            d_existente = get_disciplina(disciplinas, d_id)
            # Mesmo professor no mesmo horário → conflito
            if d_existente["prof"] == disc["prof"]:
                return False
    
    return True


# Contador global de nós expandidos (para métricas)
nos_expandidos = 0


def gerar_sucessores(estado, disciplinas, salas, horarios):
    """
    Gera estados sucessores a partir do estado atual.
    Estratégia: escolher a disciplina não alocada com mais alunos (mais difícil).
    """
    global nos_expandidos
    nos_expandidos += 1  # contamos uma expansão de nó
    
    sucessores = []
    nao_alocadas = disciplinas_nao_alocadas(estado, disciplinas)
    
    if not nao_alocadas:
        return sucessores
    
    # Heurística simples: ordenar pela quantidade de alunos (decrescente)
    disc = sorted(nao_alocadas, key=lambda d: -d["alunos"])[0]
    
    for sala in salas:
        for horario in horarios:
            if eh_alocacao_valida(estado, disciplinas, salas, disc["id"], sala["id"], horario):
                novo_estado = dict(estado)
                novo_estado[disc["id"]] = (sala["id"], horario)
                sucessores.append(novo_estado)
    
    return sucessores


## Busca sem informação (híbrido BFS + DFS)

Nesta parte, vamos implementar uma estratégia **sem informação**:

- **Fase 1 – BFS (Busca em Largura):**
  - Explora alguns níveis iniciais da árvore de busca em largura.
  - Objetivo: gerar configurações iniciais variadas (vários jeitos de começar a alocação).

- **Fase 2 – DFS (Busca em Profundidade com backtracking):**
  - A partir dos estados coletados na BFS, usamos DFS recursiva para tentar completar a alocação.
  - Se um caminho dá conflito, voltamos atrás (backtracking).

Objetivo principal da busca sem informação:
> Encontrar uma solução **sem usar heurística** sobre qual estado é “melhor”.


In [3]:
from collections import deque

def busca_hibrida_sem_informacao(estado_inicial, disciplinas, salas, horarios, nivel_bfs=2):
    """
    Estratégia sem informação:
      1) BFS até um certo nível (nivel_bfs).
      2) Para cada estado coletado na fronteira, aplicar DFS recursivo.
    """
    # -----------------------
    # FASE 1: BFS até nivel_bfs
    # -----------------------
    fila = deque([(estado_inicial, 0)])  # (estado, profundidade)
    estados_bfs = []
    
    while fila:
        estado, prof = fila.popleft()
        
        if prof == nivel_bfs:
            estados_bfs.append(estado)
            continue
        
        sucessores = gerar_sucessores(estado, disciplinas, salas, horarios)
        for s in sucessores:
            fila.append((s, prof + 1))
    
    # -----------------------
    # FASE 2: DFS com backtracking
    # -----------------------
    visitados = set()
    
    def dfs(estado):
        # Teste de objetivo: todas as disciplinas alocadas
        if len(estado) == len(disciplinas):
            return estado
        
        chave = tuple(sorted(estado.items()))
        if chave in visitados:
            return None
        visitados.add(chave)
        
        for s in gerar_sucessores(estado, disciplinas, salas, horarios):
            sol = dfs(s)
            if sol is not None:
                return sol
        return None
    
    for e in estados_bfs:
        sol = dfs(e)
        if sol is not None:
            return sol
    
    return None


## Busca com informação (heurística)

Agora vamos usar uma **heurística** para guiar a busca.

Exemplos de heurística possíveis:
- `h(estado) = número de disciplinas que ainda faltam alocar`.
- Ou somar o número de alunos das disciplinas que faltam (disciplinas grandes são mais difíceis de encaixar).

Aqui vamos usar uma heurística simples:

> **h(estado) = quantidade de disciplinas não alocadas**

E vamos implementar uma **busca gulosa**:
- Sempre escolhe expandir o estado com o menor valor de `h(estado)`.
- Isso tenta ir mais rápido para estados “quase completos”.


In [6]:
import heapq
import itertools  # <-- importa o contador

def heuristica_num_restantes(estado, disciplinas):
    """Heurística: número de disciplinas que ainda faltam ser alocadas."""
    return len(disciplinas_nao_alocadas(estado, disciplinas))


def busca_gulosa(estado_inicial, disciplinas, salas, horarios):
    """
    Busca gulosa:
      - A fronteira é uma fila de prioridade ordenada por h(estado).
      - Sempre expandimos o estado com menor valor de h.
    """
    global nos_expandidos
    nos_expandidos = 0

    # contador para gerar IDs únicos e evitar comparar dicionários no heap
    contador = itertools.count()

    fila = []
    h0 = heuristica_num_restantes(estado_inicial, disciplinas)
    heapq.heappush(fila, (h0, next(contador), estado_inicial))
    visitados = set()

    while fila:
        h, _, estado = heapq.heappop(fila)

        # Teste de objetivo
        if len(estado) == len(disciplinas):
            return estado

        chave = tuple(sorted(estado.items()))
        if chave in visitados:
            continue
        visitados.add(chave)

        sucessores = gerar_sucessores(estado, disciplinas, salas, horarios)
        for s in sucessores:
            h_s = heuristica_num_restantes(s, disciplinas)
            # agora a tupla tem: (heurística, id_único, estado)
            heapq.heappush(fila, (h_s, next(contador), s))

    return None


## Comparação das estratégias

Vamos rodar:

- A busca gulosa (com informação).
- A busca híbrida BFS + DFS (sem informação).

E para cada uma vamos medir:

- Número de nós expandidos.
- Tempo de execução.
- A solução encontrada (alocação de disciplinas em salas/horários).


In [7]:
import time

def rodar_busca(func_busca, descricao, *args, **kwargs):
    """Executa uma função de busca, mede tempo e nós expandidos."""
    global nos_expandidos
    nos_expandidos = 0
    
    inicio = time.time()
    solucao = func_busca(*args, **kwargs)
    fim = time.time()
    
    tempo = fim - inicio
    nos = nos_expandidos
    
    print(f"=== {descricao} ===")
    print(f"Solução encontrada: {solucao is not None}")
    print(f"Nós expandidos: {nos}")
    print(f"Tempo (s): {tempo:.6f}")
    print()
    
    return solucao, nos, tempo


# Rodando a busca gulosa (com informação)
sol_gulosa, nos_g, tempo_g = rodar_busca(
    busca_gulosa,
    "Busca Gulosa (com heurística)",
    estado_inicial,
    disciplinas,
    salas,
    horarios
)

# Rodando a busca híbrida sem informação
sol_hibrida, nos_h, tempo_h = rodar_busca(
    busca_hibrida_sem_informacao,
    "Busca Híbrida BFS+DFS (sem informação)",
    estado_inicial,
    disciplinas,
    salas,
    horarios,
    nivel_bfs=2
)


=== Busca Gulosa (com heurística) ===
Solução encontrada: True
Nós expandidos: 5
Tempo (s): 0.000112

=== Busca Híbrida BFS+DFS (sem informação) ===
Solução encontrada: True
Nós expandidos: 7
Tempo (s): 0.000113



In [8]:
def imprimir_solucao(solucao):
    if solucao is None:
        print("Nenhuma solução encontrada.")
        return
    
    print("Alocação de disciplinas (disciplina → sala, horário):")
    for disc_id, (sala_id, horario) in solucao.items():
        disc = get_disciplina(disciplinas, disc_id)
        print(f"- {disc_id} ({disc['prof']}, {disc['alunos']} alunos) → Sala {sala_id}, Horário {horario}")


print("=== Solução da busca gulosa ===")
imprimir_solucao(sol_gulosa)
print("\n=== Solução da busca híbrida ===")
imprimir_solucao(sol_hibrida)


=== Solução da busca gulosa ===
Alocação de disciplinas (disciplina → sala, horário):
- IA (Lucidio, 35 alunos) → Sala S1, Horário 08-10
- BD (Maria, 28 alunos) → Sala S1, Horário 08-10
- PO (Maria, 25 alunos) → Sala S1, Horário 10-12
- ALC (Joao, 20 alunos) → Sala S1, Horário 08-10
- CG (Ana, 18 alunos) → Sala S1, Horário 08-10

=== Solução da busca híbrida ===
Alocação de disciplinas (disciplina → sala, horário):
- IA (Lucidio, 35 alunos) → Sala S1, Horário 08-10
- BD (Maria, 28 alunos) → Sala S1, Horário 08-10
- PO (Maria, 25 alunos) → Sala S1, Horário 10-12
- ALC (Joao, 20 alunos) → Sala S1, Horário 08-10
- CG (Ana, 18 alunos) → Sala S1, Horário 08-10


## Conclusão

Neste notebook, modelamos o **problema de alocação de salas** como um problema de busca em espaço de estados:

- Cada estado representa uma alocação parcial ou completa de disciplinas em salas e horários.
- Definimos restrições de capacidade de sala e conflitos de professor.

Comparando as estratégias:

- **Busca sem informação (BFS + DFS)**:
  - Explora o espaço de forma mais “cego”, sem usar heurística.
  - Pode expandir mais nós e demorar mais, dependendo da instância.

- **Busca com informação (busca gulosa com heurística)**:
  - Usa uma estimativa (`número de disciplinas restantes`) para decidir qual estado expandir.
  - Em geral, tende a chegar mais rápido em uma solução (menos nós expandidos).

Limitações do trabalho:
- Usamos um exemplo pequeno de disciplinas, salas e horários.
- Não consideramos todas as restrições possíveis (conflitos entre alunos, prédios diferentes, etc.).

Possíveis melhorias:
- Testar outras heurísticas mais sofisticadas.
- Implementar A* (f = g + h) em vez de apenas busca gulosa.
- Aumentar a quantidade de dados para aproximar melhor a realidade da alocação de salas da UFPB.
